In [ ]:
! make clean

In [ ]:
! make clean-logs

In [ ]:
! rm  -rf /media/sayem/510B93E12554BBD1/Hangman/wandb
! rm -rf /media/sayem/510B93E12554BBD1/checkpoints 

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed):
    """Set seed for reproducibility."""
    random.seed(seed)       # Python random module
    np.random.seed(seed)    # Numpy module
    torch.manual_seed(seed) # PyTorch
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)          # Sets seed for CUDA (GPU)
        torch.cuda.manual_seed_all(seed)      # Ensure reproducibility on all GPUs
        torch.backends.cudnn.deterministic = True  # Use deterministic algorithms
        torch.backends.cudnn.benchmark = False     # If input sizes do not vary, this should be set to False

# Example usage: 
set_seed(42)  # Use any number to seed all libraries

#### Imports

In [ ]:
# from src.datamodule import HangmanDataset
from src.datamodule import HangmanDataModule
# from src.datamodule import encode_and_pad_hangman_features
# from src.datamodule import *
# from src.model.neural_nets import *
# from src.model.neural_nets.classifier import *
torch.cuda.empty_cache()

# torch.set_float32_matmul_precision('high')
import lightning as L 
# # L.seed_everything(102, workers=True)
# np.random.seed(102)  # You can use any number here

In [ ]:
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
# torch.backends.cuda.matmul.allow_tf32 = False
# # torch.use_deterministic_algorithms(True)  # Requires PyTorch >= 1.8

In [ ]:
from src.utils import read_word_list
corpus_path = '/media/sayem/510B93E12554BBD1/Hangman/data/words_250000_train.txt'
corpus = read_word_list(corpus_path, num_samples=250_000)  # corrected format for num_samples

#### Paths

In [ ]:
import shutil
from pathlib import Path
import json
from src.utils import read_word_list

NUM_STRATIFIED_SAMPLES = 250_000
word_list_path = '/media/sayem/510B93E12554BBD1/Hangman/notebooks/api_key.txt'
word_list = read_word_list(word_list_path, num_samples=NUM_STRATIFIED_SAMPLES)

# Base dataset directory and subdirectories
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")
# parquet_path = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)

In [ ]:
# loss = -[w_i * (y_i * pos_weight * log(sigma(x_i)) + (1 - y_i) * log(1 - sigma(x_i)))]

#### Neural Net

In [ ]:
feature_extractor_params = {
    "vocab_size": 28,               # Non-tunable: Set based on the dataset specifics (number of unique characters)
    "embedding_dim": 50,           # Tunable: Size of the embedding vectors
    "hidden_dim": 128,              # Tunable: Hidden layer size in LSTM
    "num_layers": 5,                # Tunable: Number of layers in the LSTM
    "bidirectional": True,          # Tunable: Whether the LSTM is bidirectional
    "dropout": 0.3,                 # Tunable: Dropout rate to prevent overfitting
    "max_norm": 1,                  # Tunable: Maximum norm for gradient clipping
    "feature_dim": 5                # Non-tunable/Tunable: Depending on whether 
                                    # this is a fixed architectural choice or could be optimized
}

# # Assuming the EmbeddingLSTM class is defined somewhere and imported
# # Instantiate the feature extractor with parameters unpacked from the dictionary
# feature_extractor = EmbeddingLSTM(**feature_extractor_params)

# Assuming `feature_extractor_params` has keys 'hidden_dim' and 'bidirectional'
input_dim = feature_extractor_params['hidden_dim'] * 2 if \
    feature_extractor_params.get('bidirectional', False) else feature_extractor_params['hidden_dim']

classifier_params = {
    "input_dim": input_dim,         # Non-tunable: Computed based on 'hidden_dim' from LSTM and whether it is bidirectional
    "output_dims": 26,              # Non-tunable: Set based on the number of labels (assumes fixed number of classes)
    "initial_biases": None,  # Tunable: Could be optimized if linked to class imbalance
    "hidden_dims": [256, 128, 64],   # Tunable: Sizes of additional hidden layers in the classifier
    "dropout": 0.3,                 # Tunable: Dropout rate for regularization
    "norm_layer_func": torch.nn.LayerNorm,  # Tunable: Type of normalization layer can affect model performance
    "activation_layer": torch.nn.ReLU,  # Tunable: Choice of activation function can influence model dynamics
    # "randomize": True              # Non-tunable/Tunable: Typically a methodological choice, not a hyperparameter
}

# # Initialize the classifier with parameters unpacked from the dictionary
# classifier = DynamicClassifierChain(**classifier_params)

In [ ]:
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# Example optimizer and scheduler configuration
optimizer_config = {
    'type': Adam,
    'params': {'lr': 0.001, 'betas': (0.9, 0.999)}
}

# from torch.optim import SGD

# # Example optimizer configuration using SGD
# optimizer_config = {
#     'type': SGD,
#     'params': {'lr': 0.01, 'momentum': 0.9}
# }

scheduler_config = {
    'type': ReduceLROnPlateau,
    'params': {
        'mode': 'max',  # or 'min' depending on the nature of the metric to monitor
        'factor': 0.1,  # Factor by which the learning rate will be reduced. new_lr = lr * factor
        'patience': 10,  # Number of epochs with no improvement after which learning rate will be reduced.
        'threshold': 0.01,  # Minimal change to qualify as an improvement.
        'threshold_mode': 'rel',  # 'rel' implies relative change, 'abs' implies absolute change.
        'cooldown': 0,  # Number of epochs to wait before resuming normal operation after lr has been reduced.
        'min_lr': 0,  # A lower bound on the learning rate of all param groups or each group respectively.
        'eps': 1e-08,  # Minimal decay applied to lr. If the difference between new and old lr is smaller than eps, the update is ignored.
        # 'verbose': True  # If True, prints a message to stdout for each update.
    },
    'interval': 'epoch',  # 'ReduceLROnPlateau' typically does not need the 'interval' and 'frequency' since it works based on metric change.
    'frequency': 1
}

In [ ]:
from src.model import HangmanModel
from src.model.components import EmbeddingLSTM, LabelEmbeddingNN, BinaryRelevanceClassifier
from src.data_generation.strategy import HangmanFreqSolver # strategy
# from src.model.components import GuessAccuracy
from pytorchltr.loss import PairwiseHingeLoss, LambdaARPLoss1, \
    LambdaARPLoss2, LambdaNDCGLoss2, PairwiseDCGHingeLoss, PairwiseLogisticLoss

solver = HangmanFreqSolver(corpus)
# metric = GuessAccuracy(solver=solver)
criterion = PairwiseLogisticLoss(sigma=1.0)
# criterion = PairwiseHingeLoss() # thresholds = torch.full((26,), 0.5)
# print(thresholds)
model = HangmanModel(
                     feature_extractor=EmbeddingLSTM, 
                     feature_extractor_params=feature_extractor_params,
                     classifier=BinaryRelevanceClassifier, #LabelEmbeddingNN,
                     classifier_params=classifier_params,
                     optimizer_config=optimizer_config,
                     scheduler_config=scheduler_config,
                     pos_weight = None
                    #  critarion=criterion
)
# model = model.to('cuda')
# model.device

In [ ]:
# for i, (train_dataset, val_dataset) in enumerate(fold_datasets):
#     train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, 
#                                   num_workers=os.cpu_count(), collate_fn=datamodule.collate_fn)

#     val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False, 
#                                 num_workers=os.cpu_count(), collate_fn=datamodule.collate_fn)

In [ ]:
# checkpoint_path = 'checkpoints/best-checkpoint.ckpt'
# checkpoint = torch.load(checkpoint_path)
# hyper_parameters = checkpoint["hyper_parameters"]

In [ ]:
# hyper_parameters = checkpoint["hyper_parameters"]
# # if you want to restore any hyperparameters, you can pass them too
# # model = HangmanModel(critarion=criterion, **hyper_parameters)
# model = HangmanModel(**hyper_parameters)
# model_weights = checkpoint["state_dict"]
# model.load_state_dict(model_weights)
# model.eval()

In [ ]:
# model = model.to('cpu')
# trainer.validate(model, datamodule)

#### Game Simulation

- Generating action

In [ ]:
## load the model from checkpoint

In [ ]:
from src.data_generation.simulation import play_a_game_with_a_word, \
    simulate_games_for_word_list # testing function
from src.datamodule.transforms import ProcessWordTransform
from src.data_generation.strategy import HangmanFreqSolver # strategy
from src.data_generation.process_word import process_word
from src.datamodule.dataset import encode_character

from src.model.inference import guess, guess_character
from src.datamodule.transforms import ProcessWordTransform
# from src.model.strategy import HangmanFreqSolver # strategy
from src.data_generation.simulation import play_a_game_with_a_word, \
                            simulate_games_for_word_list # testing function
from src.data_generation.data_generation \
    import simulated_guess_function, generate_a_game_with_a_word

solver = HangmanFreqSolver(corpus) # TODO: does it matter waht corpus, since no use use in guess?
transform = ProcessWordTransform(corpus) # here, what corpus does not matter

# Example word
# # real_word = 'aaup' 
# word = 'mississippi' # out of corpus
# word = 'zyg' # from corpus
word = 'microspace'
# # real_word = 'ask'
# masked_word = "_" * len(real_word)
# # masked_word = "aa__"
# guessed_letters = []
# word = 'apple'
masked_word = '_' * len(word)
guessed_letters = []

In [ ]:
# strategy = 'min'

# simulated_guess_function(word, masked_word, \
#             solver, guessed_letters, strategy)

In [ ]:
# strategy = 'min'
# # from src.data_generation.data_generation import simulated_guess_function # same as guess+ guess character function
# generate_a_game_with_a_word(word=word, \
#             guess_function=simulated_guess_function, \
#             solver=solver, strategy=strategy) 

In [ ]:
# strategy = 'max'
# # from src.data_generation.data_generation import simulated_guess_function # same as guess+ guess character function
# # generate_a_game_with_a_word(word=word, \
#         guess_function=simulated_guess_function, \
#         solver=solver, strategy=strategy) 

In [ ]:
# strategy = 'random'
# # from src.data_generation.data_generation import simulated_guess_function # same as guess+ guess character function
# generate_a_game_with_a_word(word=word, \
#         guess_function=simulated_guess_function, \
#         solver=solver, strategy=strategy) 

In [ ]:
# strategy = 'random_420'
# # from src.data_generation.data_generation import simulated_guess_function # same as guess+ guess character function
# generate_a_game_with_a_word(word=word, \
#         guess_function=simulated_guess_function, \
#         solver=solver, strategy=strategy) 

In [ ]:
# from src.data_generation.data_generation import simulated_guess_function, generate_a_game_with_a_word
# # model = model.to('cuda')
# # # Example usage
# # solver = HangmanFreqSolver(corpus)
# corpus_path_ = 'data/20k.txt'
# word_list = read_word_list(corpus_path_, num_samples=1_000)
# final_results = simulate_games_for_word_list(word_list=word_list, guess_function=simulated_guess_function, \
#                                             play_function=generate_a_game_with_a_word, \
#                                             model=None, solver=solver, \
#                                             transform=transform, process_word_fn=process_word) 

# # Print overall statistics
# overall_stats = final_results['overall']
# print("\nOverall Statistics:")
# print(f"Total Games: {overall_stats['total_games']}, Wins: {overall_stats['wins']}, Losses: {overall_stats['losses']}")
# print(f"Win Rate: {overall_stats['win_rate']:.2f}, Average_tries_remaining: {overall_stats['average_tries_remaining']:.2f}")

- Motoring:

In [ ]:
from src.model.inference import guess, guess_character
from src.datamodule.transforms import ProcessWordTransform
# from src.model.strategy import HangmanFreqSolver # strategy
from src.data_generation.simulation import play_a_game_with_a_word, \
                            simulate_games_for_word_list # testing function

result = process_word(word, transform)
print(result)
real_word = 'aarau'
# real_word = 'mississippi' # out of corpus
# real_word = 'zyg' # from corpus"
# real_word = 'ask'
masked_word = "_" * len(real_word)
# masked_word = "aa__"
guessed_letters = []

In [ ]:
guess(model=model, word=masked_word, # TODO
    solver=solver, \
    guessed_letters=guessed_letters, 
    transform=transform,
    process_word_fn=process_word) 

In [ ]:
from src.env import HangmanEnv

# Usage Example
# if __name__ == "__main__":
# words = ["python", "algorithm", "function", "variable"]
env = HangmanEnv("example")
print("Initial state:", env.reset())
# print()
# action = 'e'  # Guessing 'e'
# observation, reward, done = env.step(action)
# print("Observation:", observation)
# print("Reward:", reward)
# print("Done:", done)

In [ ]:
env.action_masks()

In [ ]:
from sb3_contrib.common.maskable.utils import get_action_masks

from sb3_contrib.common.maskable.utils import get_action_masks, is_masking_supported

# action_masks = get_action_masks(vec_env)
is_masking_supported(env)

In [ ]:
get_action_masks(env)

In [ ]:
from sb3_contrib.common.maskable.utils import get_action_masks

if __name__ == "__main__":
    env = HangmanEnv("ant")
    observation, info = env.reset()
    done = False
    while not done:
        print('Observation:', observation)
        action = env.action_space.sample()
        # print(action)

        action_mask = get_action_masks(env)
        print(f"What can predict:", action_mask)
        
        # guessed_letters = observation['guessed_letters']
        # # print(guessed_letters)
        # # # Convert action to corresponding letter and check if it has been guessed
        while not action_mask[action]:  # Check if the action index (converted to a letter) is already guessed
            print(f"resampling")
            action = env.action_space.sample()  # Keep sampling until a new letter is found

        print('Action: ', action)
        # Perform the action in the environment
        next_observation, reward, done, truncated, info = env.step(action)
        
        print('Reward:', reward)
        print('Next observation:', next_observation)
        print('info:', info)
        print('done:', done)
        print()

        # Update the initial observation for the next loop iteration
        observation = next_observation
        # break

        # Optionally, env.render() if implemented

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

In [ ]:
def env_creator(word, **kwargs):
    """ Returns a function that will initialize the Hangman environment with given parameters """
    def _init():
        return HangmanEnv(word=word, **kwargs)
    return _init


words = ["apple", "ant", "zgy"]
env_functions = [env_creator(word) for word in words]

# # Using DummyVecEnv for simplicity and synchronicity
vec_env = DummyVecEnv(env_fns=env_functions)
# Using DummyVecEnv for simplicity and synchronicity
# vec_env = SubprocVecEnv(env_fns=env_functions)

In [ ]:
vec_env.reset()

In [ ]:
from sb3_contrib.common.maskable.utils import get_action_masks, is_masking_supported

# action_masks = get_action_masks(vec_env)
is_masking_supported(vec_env)

In [ ]:
get_action_masks(vec_env)

In [ ]:
vec_env.reset()
actions = [vec_env.action_space.sample() for _ in range(len(words))]  # Ensure an action is sampled for each environment
print(actions)
vec_env.step(actions)  # Apply batch of actions

In [ ]:
vec_env.num_envs

In [ ]:
from sb3_contrib.common.maskable.policies import MaskableMultiInputActorCriticPolicy
from sb3_contrib.ppo_mask import MaskablePPO
import gymnasium as gym
from gymnasium import spaces
from typing import Callable, Dict, List, Optional, Tuple, Type, Union

from stable_baselines3 import PPO
model = MaskablePPO(MaskableMultiInputActorCriticPolicy, vec_env, verbose=1)
model.learn(total_timesteps=10_000)

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback

# def make_env(word):
#     def _init():
#         return HangmanEnv(word=word)
#     return _init

def env_creator(word, **kwargs):
    """ Returns a function that will initialize the Hangman environment with given parameters """
    def _init():
        return HangmanEnv(word=word, **kwargs)
    return _init


# # words = ["apple", "ant", "zgy"]
# env_functions = [env_creator(word) for word in words]


# # # Batch processing and training
# corpus = ["word", "word", "word"] # , ..., "word250000"]  # Actual list of words
batch_size = 4  # Words per batch
model = MaskablePPO(MaskableMultiInputActorCriticPolicy, \
            make_vec_env(lambda: HangmanEnv("placeholder"), n_envs=batch_size), verbose=1)

for i in range(0, len(corpus), batch_size):
    batch_words = corpus[i:i + batch_size]
    # print(len(batch_words))
    env_fns = [env_creator(word) for word in batch_words]
    # # vec_env = DummyVecEnv(env_fns)  # Create vectorized environment for the batch
    vec_env = SubprocVecEnv(env_fns)
    model.set_env(vec_env)  # Set the new environment to the model
    model.learn(total_timesteps=10_000)  # Learning on the batch for a fixed number of timesteps

    # # # Optionally, evaluate the model's performance on the batch
    # # eval_callback = EvalCallback(vec_env, best_model_save_path='./logs/',
    # #                              log_path='./logs/', eval_freq=500,
    # #                              deterministic=True, render=False)
    # # model.learn(total_timesteps=10000, callback=eval_callback)

    vec_env.close()  # Close the environment after training on the batch
    
    if i + batch_size >= len(corpus):  # Check if this is the last batch
        print("Last batch processed. Exiting loop.")
        break  # Exit the loop after the last batch

In [ ]:
STOP

In [ ]:
# observations = vec_env.reset()

# dones = np.zeros(len(words), dtype=bool)
# max_timesteps = 30
# timestep = 0

# while timestep < max_timesteps:
#     print(f'Timestep {timestep + 1}:')
#     print('Observations:', observations)
    
#     actions = []
#     active_envs = 0  # Count active environments for this pass

#     for i in range(len(words)):
#         action = vec_env.action_space.sample()  # Randomly sample an action
#         while observations['guessed_letters'][i][action] == 1:
#             action = vec_env.action_space.sample()  # Ensure this action hasn't been taken before
#         observations['guessed_letters'][i][action] = 1  # Mark the action as taken
#         actions.append(action)
#         active_envs += 1  # Count this as an active interaction

#     print('Actions:', [chr(action + ord('a')) for action in actions])
#     observations, rewards, dones, info = vec_env.step(actions)  # Apply batch of actions

#     print('Rewards:', rewards)
#     print('Infos:', info)
#     print()

#     timestep += active_envs  # Increment the timestep by the number of active environments

#     # if all(dones):
#     #     print('All games completed.')
#     #     break

In [ ]:
# observations = vec_env.reset()

# # Loop through the environments
# while True:
#     actions = [vec_env.action_space.sample() for _ in range(len(words))]  # Ensure an action is sampled for each environment
#     print(actions)
#     observations, rewards, dones, truncated = vec_env.step(actions)  # Apply batch of actions
    
#     print('Actions:', [chr(action + ord('a')) for action in actions])
#     print('Observations:', observations)
#     print('Rewards:', rewards)
#     print()
    
#     if all(dones):
#         print('Game Over for all environments')
#         break
#     print()

In [ ]:
# model.optimal_threshold = torch.full((26,), 0.9) # thresholds

In [ ]:
play_a_game_with_a_word(word=real_word, guess_function=guess, \
                model=model, solver=solver, transform=transform, \
                process_word_fn=process_word) # aggregated_data=None): # TODO: aggregated_data=None: remove later

In [ ]:
STOP

In [ ]:
play_a_game_with_a_word(word=real_word, guess_function=guess, \
                model=model, solver=solver, transform=transform, \
                process_word_fn=process_word) # aggregated_data=None): # TODO: aggregated_data=None: remove later

In [ ]:
# model.hparams['thresholds'] = torch.full((26,), 1) #datamodule.base_rate #torch.full((26,), 0.1) # thresholds

In [ ]:
# model = model.to('cuda')
# # # Example usage
# # solver = HangmanFreqSolver(corpus)
# corpus_path_ = '/media/sayem/510B93E12554BBD1/Hangman/data/20k.txt'

# word_list = read_word_list(corpus_path_, num_samples=1_000)
# final_results = simulate_games_for_word_list(word_list=word_list, guess_function=guess, \
#                                             play_function=play_a_game_with_a_word, \
#                                             model=model, solver=solver, \
#                                             transform=transform, process_word_fn=process_word) 

# # Print overall statistics
# overall_stats = final_results['overall']
# print("\nOverall Statistics:")
# print(f"Total Games: {overall_stats['total_games']}, Wins: {overall_stats['wins']}, Losses: {overall_stats['losses']}")
# print(f"Win Rate: {overall_stats['win_rate']:.2f}, Average_tries_remaining: {overall_stats['average_tries_remaining']:.2f}")

In [ ]:
# model = model.to('cuda')
# # # Example usage
# # solver = HangmanFreqSolver(corpus)
# corpus_path_ = 'data/20k.txt'
# word_list = read_word_list(corpus_path_, num_samples=1_0)
# final_results = simulate_games_for_word_list(word_list=word_list, guess_function=guess, \
#                                             play_function=play_a_game_with_a_word, \
#                                             model=model, solver=solver, \
#                                             transform=transform, process_word_fn=process_word, guessing_order=None) 

# # Print overall statistics
# overall_stats = final_results['overall']
# avg_tries_remaining = overall_stats['average_tries_remaining']
# print(avg_tries_remaining)
# print("\nOverall Statistics:")
# print(f"Total Games: {overall_stats['total_games']}, Wins: {overall_stats['wins']}, Losses: {overall_stats['losses']}")
# print(f"Win Rate: {overall_stats['win_rate']:.2f}, Average_tries_remaining: {overall_stats['average_tries_remaining']:.2f}")

In [ ]:
# import optuna

# def multi_objective_trial(trial):
#     # Define the range for each threshold as a trial suggestion
#     thresholds = [trial.suggest_float(f'threshold_{i}', 0, 1) for i in range(26)]
#     thresholds_tensor = torch.tensor(thresholds, dtype=torch.float, device='cuda')
#     model.hparams['thresholds'] = thresholds_tensor
#     corpus_path_ = 'data/20k.txt'
#     word_list = read_word_list(corpus_path_, num_samples=1_000)
        
#     # Run the game simulation
#     final_results = simulate_games_for_word_list(word_list=word_list, guess_function=guess, 
#                                                  play_function=play_a_game_with_a_word, 
#                                                  model=model, solver=solver, 
#                                                  transform=transform, process_word_fn=process_word)
    
#     # Return multiple objectives: win rate and negative average tries remaining (to maximize)
#     win_rate = final_results['overall']['win_rate']
#     avg_tries_remaining = final_results['overall']['average_tries_remaining']
#     return win_rate, avg_tries_remaining

# # def main():
# # Create a multi-objective study
# study = optuna.create_study(directions=['maximize', 'maximize'])
# # study = optuna.create_study(directions=['maximize'])

# # Execute optimization
# study.optimize(multi_objective_trial, n_trials=3)

# # Print results
# print("Best trials:")
# for trial in study.best_trials:
#     print(f"  Win Rate: {trial.values[0]}")
#     print(f"  Average Tries Remaining: {trial.values[1]}")
#     print(f"  Thresholds: {trial.params}")

# # if __name__ == "__main__":
# #     main()

#### API Integration

In [ ]:
from src.api import HangmanAPI
import time

In [ ]:
# Specify the path to the file
file_path = '/media/sayem/510B93E12554BBD1/Hangman/notebooks/api_key.txt'

# Use a context manager to open and read the file
with open(file_path, 'r') as file:
    api_key = file.read().strip()  # Read the content and strip any extra whitespace

# print("API Key:", api_key)

In [ ]:
api = HangmanAPI(model=model, corpus_path=corpus_path, \
                solver=solver, transform=transform, \
                access_token=api_key, process_word_fn=process_word, timeout=2_000)

In [ ]:
api.start_game(practice=1, verbose=True)
[total_practice_runs, total_recorded_runs,total_recorded_successes, total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

In [ ]:
STOP

In [ ]:
[total_practice_runs, total_recorded_runs, \
            total_recorded_successes, total_practice_successes] = api.my_status()

In [ ]:
[total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] 

In [ ]:
for i in range(100):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    # api.start_game(practice=0,verbose=False)
    api.start_game(practice=1, verbose=False)
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

In [ ]:
[total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes]